# Monitoring COVID-19 Infections through Wastewater Surveillance
### Authors (perm, email): Romtin Toranji (5235684 rtoranji@ucsb.edu), Matthew Coleman (5267398 matthewbcoleman@ucsb.edu) Andrew Nguyen (5495395 ahn00@ucsb.edu)


In [1]:
import numpy as np

In [10]:
one_a = np.load('part1a.npz')
one_b = np.load('part1b.npz')

In [11]:
print(one_a.files)
print(one_b.files)

['N', 'Svc_0_pmf', 'Lc', 'Ic_0', 'gamma']
['N', 'Lc', 'Ic_0', 'gamma', 'L_validation', 'L_test', 'betas_validation']


In [12]:
print(one_a['N'])
print(one_b['N'])

100000.0
50000.0


In [17]:
print(one_a['Ic_0'])

[47 90 51 65]
